In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
# Import utilities
from setup_utils import setup_db_connection, plr, save_intermediate_data

# Connect to DB and load data
engine = setup_db_connection()

In [176]:
subject_id = 10020187

# 10020187 - 2 hadm_id, +7 unique chart dt, null height
# 10002760 - 1 hadm_id, 2 unique chart dt, has height

In [240]:
# Init subject-based dataframes

# dte cols include hadm_id/stay_id, easily join with age/height/weight tables
# vitals does not have hadm_id, so generally easier to combine after cleaning vitals data
dte_df = pd.read_sql_query(f"""
SELECT t1.subject_id, t1.hadm_id, t1.stay_id, t1.caregiver_id, t1.charttime,
    anchor_age as age,
    weight,
    height
FROM mimiciv.mimiciv_icu.datetimeevents t1
JOIN mimiciv_derived.age t2 ON t1.hadm_id = t2.hadm_id
JOIN mimiciv_derived.weight_durations t3 ON t1.stay_id = t3.stay_id
LEFT JOIN mimiciv_derived.height t4 ON t1.stay_id = t4.stay_id
WHERE t1.subject_id = {subject_id}
GROUP BY t1.subject_id, t1.hadm_id, t1.stay_id, t1.caregiver_id, t1.charttime, anchor_age, weight, height;
""", engine)

v_df = pd.read_sql_query(f"""
SELECT subject_id, stay_id, charttime, heart_rate, sbp, dbp, mbp, resp_rate, temperature, spo2
FROM mimiciv.mimiciv_derived.vitalsign
WHERE subject_id = {subject_id};
""", engine)

rx_df = pd.read_sql_query(f"""
SELECT subject_id, hadm_id, pharmacy_id, starttime 
FROM mimiciv.mimiciv_hosp.prescriptions
WHERE subject_id = {subject_id};
""", engine)

diag_df = pd.read_sql_query(f"""
SELECT * 
FROM mimiciv.mimiciv_hosp.diagnoses_icd
WHERE subject_id = {subject_id};
""", engine)

In [241]:
# Vitals
# List of vital columns to check for NaN
vital_cols = ['heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature', 'spo2']

# Problem: Multiple records can be entered in quick, complimentary succession, mirrors standard practice of collecting vitals. This spikes frequency of NaN, and increases row count.
# Solution: Join records within an established threshold into single row, dropping those which are completely NaN
# Outcome: Improves data quality, reduce frequency of NaN, reduce overall row counts

def consolidate_and_drop_empty_rows(df, time_threshold='13min'):
    """
    Consolidate sequential rows with close charttime values into a single entry,
    and drop rows where all vital columns are NaN.
    
    Parameters:
        df (pd.DataFrame): The input dataframe.
        time_threshold (str): Time difference threshold (e.g., '15min').
        
    Returns:
        pd.DataFrame: A dataframe with consolidated rows.
    """
    # Ensure charttime is a datetime object
    df['charttime'] = pd.to_datetime(df['charttime'])

    # Sort by charttime
    df = df.sort_values('charttime').reset_index(drop=True)

    
    # Initialize a consolidated dataframe
    consolidated_records = []
    current_record = df.iloc[0].copy()

    for i in range(1, len(df)):
        time_diff = df.loc[i, 'charttime'] - current_record['charttime']
        # Check if the current record has all vital columns as NaN
        
        
        # If the current row is within the time threshold, merge data
        if time_diff <= pd.to_timedelta(time_threshold):
            for col in df.columns:
                if col not in ['subject_id', 'stay_id', 'charttime']:  # Preserve key columns
                    # Take the first non-null value
                    if pd.isna(current_record[col]) and not pd.isna(df.loc[i, col]):
                        current_record[col] = df.loc[i, col]
        else:
            if not current_record[vital_cols].isna().all():            
                # Append the completed record to the results
                consolidated_records.append(current_record)
            # Start a new record
            current_record = df.iloc[i].copy()

    # Append the last record if it has at least one non-NaN vital column
    if not current_record[vital_cols].isna().all():
        consolidated_records.append(current_record)

    # Convert the list of records back to a DataFrame
    consolidated_df = pd.DataFrame(consolidated_records)
    return consolidated_df


print('original vitals df:', v_df.shape, '\nnulls:\n', v_df.isnull().sum()[v_df.isnull().sum() > 0])
v_df = consolidate_and_drop_empty_rows(v_df)
print('\ncleaned vitals df:', v_df.shape, '\nnulls:\n', v_df.isnull().sum()[v_df.isnull().sum() > 0])


original vitals df: (200, 10) 
nulls:
 heart_rate      66
sbp             69
dbp             69
mbp             68
resp_rate       68
temperature    161
spo2            70
dtype: int64

cleaned vitals df: (139, 10) 
nulls:
 heart_rate       9
sbp             12
dbp             12
mbp             11
resp_rate       11
temperature    100
spo2            13
dtype: int64


In [242]:
v_df['charttime'] = pd.to_datetime(v_df['charttime'])
v_df.set_index('charttime', inplace=True)
daily_avg_vitals = v_df[vital_cols].resample('D').mean().dropna(subset=vital_cols).reset_index()
weekly_avg_vitals = v_df[vital_cols].resample('W').mean().dropna(subset=vital_cols).reset_index()
monthly_avg_vitals = v_df[vital_cols].resample('ME').mean().dropna(subset=vital_cols).reset_index()
stay_avg_vitals = v_df.groupby('stay_id')[vital_cols].mean().reset_index()

In [243]:
daily_avg_vitals

,charttime,heart_rate,sbp,dbp,mbp,resp_rate,temperature,spo2
0,2169-01-15,74.157895,124.710526,66.868421,85.052632,16.684211,36.951667,96.210526
1,2169-01-16,65.045455,121.181818,61.477273,90.159091,17.380952,36.768333,95.454545
2,2169-01-17,61.150000,131.100000,68.750000,91.700000,17.400000,36.967143,94.850000
3,2169-01-18,63.823529,149.529412,81.588235,106.176471,16.705882,36.806667,96.235294
4,2169-01-19,69.722222,136.117647,84.176471,101.764706,16.631579,36.908333,96.277778
5,2169-01-20,66.357143,145.583333,93.666667,109.230769,18.785714,37.000000,96.428571
6,2170-02-24,67.000000,127.166667,62.333333,84.833333,14.000000,37.030000,96.400000
7,2170-02-25,60.714286,117.500000,63.285714,82.071429,14.916667,36.625000,95.181818


In [184]:
dte_df

,subject_id,hadm_id,stay_id,caregiver_id,charttime,age,weight,height
0,10020187,24104168,37509585,13754,2169-01-17 21:36:00,63,97.4,None
1,10020187,24104168,37509585,13754,2169-01-17 21:36:00,63,97.5,None
2,10020187,24104168,37509585,13754,2169-01-17 21:37:00,63,97.4,None
3,10020187,24104168,37509585,13754,2169-01-17 21:37:00,63,97.5,None
4,10020187,24104168,37509585,16457,2169-01-17 08:39:00,63,97.4,None
...,...,...,...,...,...,...,...,...
69,10020187,26842957,32554129,35263,2170-02-24 20:00:00,63,99.0,None
70,10020187,26842957,32554129,35263,2170-02-25 00:00:00,63,99.0,None
71,10020187,26842957,32554129,35263,2170-02-25 04:00:00,63,99.0,None
72,10020187,26842957,32554129,54941,2170-02-24 18:27:00,63,99.0,None


In [180]:
# get unique visits
unique_visits = dte_df[['hadm_id', 'stay_id']].drop_duplicates()

# merge rx (starttime) to unique visits; creates sequence of rx orders for each unique visit
rx_unique = rx_df.merge(unique_visits, on=['hadm_id'], how='inner')

# merge diag to unique visits; creates sequence of diagnoses events for each unique visit
diag_unique = diag_df.merge(unique_visits, on=['hadm_id'], how='inner')

# merge dte's height/weight/age into cleaned vitals

# merge stay avg vitals and unique rx
stay_vitals_and_rx = rx_unique.merge(stay_avg_vitals, on=['stay_id'], how='inner')

# merge stay avg vitals and diagnoses events
stay_vitals_and_diag = diag_unique.merge(stay_avg_vitals, on=['stay_id'], how='inner')



In [172]:
stay_vitals_and_rx

,subject_id,hadm_id,pharmacy_id,starttime,stay_id,heart_rate,sbp,dbp,mbp,resp_rate,temperature,spo2
0,10020187,24104168,6088913,2169-01-15 16:00:00,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
1,10020187,24104168,6088913,2169-01-15 16:00:00,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
2,10020187,24104168,9548642,2169-01-15 08:00:00,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
3,10020187,24104168,9548642,2169-01-15 08:00:00,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
4,10020187,24104168,11262209,2169-01-15 06:00:00,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
...,...,...,...,...,...,...,...,...,...,...,...,...
70,10020187,26842957,66301993,2170-02-24 19:00:00,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
71,10020187,26842957,66833512,2170-02-24 22:00:00,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
72,10020187,26842957,79792249,2170-02-25 08:00:00,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
73,10020187,26842957,86062207,2170-02-24 19:00:00,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500


In [171]:
stay_vitals_and_diag

,subject_id,hadm_id,seq_num,icd_code,icd_version,stay_id,heart_rate,sbp,dbp,mbp,resp_rate,temperature,spo2
0,10020187,24104168,1,I6032,10,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
1,10020187,24104168,2,I10,10,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
2,10020187,24104168,3,E785,10,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
3,10020187,24104168,4,I2510,10,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
4,10020187,24104168,5,E780,10,37509585,66.654545,133.275701,74.205607,96.189815,17.209091,36.890303,95.854545
5,10020187,26842957,1,I671,10,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
6,10020187,26842957,2,Z6841,10,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
7,10020187,26842957,3,I10,10,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
8,10020187,26842957,4,E785,10,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500
9,10020187,26842957,5,E669,10,32554129,62.600000,120.400000,63.000000,82.900000,14.611111,36.760000,95.562500


In [155]:

heart_rate_trends = stay_vitals_and_diag.groupby('icd_code')['heart_rate'].mean()

# Example: Trends in prescriptions by average blood pressure
bp_prescription_trends = stay_vitals_and_rx.groupby('pharmacy_id')['mbp'].mean()

In [167]:
heart_rate_trends

icd_code
E780       66.654545
I2510      66.654545
I6032      66.654545
E785       64.627273
I10        64.627273
E669       62.600000
I671       62.600000
K2270      62.600000
M1712      62.600000
Z6841      62.600000
Z7902      62.600000
Z96651     62.600000
Name: heart_rate, dtype: float64